# 2017-05-02 : grabbing the framerate and plotting it
----------------------------

- [X] Capturing a framerate for each downscale parameter possible

Obtaining different framerates will allow us to chose which downscale using for an optimum recording.  
The framerate seems to be hightly dependent of the program simplicity. Going from : 

    (...)
    for i in range(0,num_frames):
        Ph.grab()
        frame = Ph.grab()
        (...)
        
to :

    (...)
    for i in range(0,num_frames):
        frame = Ph.grab()
        (...)
        
Seem to double the framerate recorded (from 7 to 13 FPS). It is understandable since the first version executed Ph.grab() two times per loop. Also, the version b (limit : time recorded) seems to be a bit faster than the version a (limit : number of frames recorded) as it record at 14-15 FPS (vs 13 FPS).  
The program contained a little writing error that prevented it to work correctly when the downscale parameter was superior to 1. Going from :

    (...)
    if DOWNSCALE > 1 :
        W = self.cap.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH)
        H = self.cap.set(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT)
        (...)

to :

    (...)
    if DOWNSCALE > 1 :
        W = self.cap.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH)
        H = self.cap.get(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT)
        (...)    

I built a loop that allow to modify the downscale parameter :

    (...)
    for ds in range(10):
        Ph = PhotoReceptor(w=1280, h=720, DOWNSCALE = ds)
        (...)

In these conditions, the framerate seems to not be influenced by the downscale, as the program output the same FPS for each of them (except for downscale = 2).

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 1. dim1 : 720.0, dim2 : 1280.0
    Frame rate : 15.0 

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 2. dim1 : 288.0, dim2 : 352.0
    Frame rate : 17.0 

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 3. dim1 : 240.0, dim2 : 320.0
    Frame rate : 15.0 

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 4. dim1 : 144.0, dim2 : 176.0
    Frame rate : 15.0
    
    (...)
    
In addition, a problem appear from downscale = 7 to beyond : the new dimensions are not reduced as expected. It looks like the program is not able to reduce them this much and so it may represent the upper limit of the downscaling in these conditions.  

    (...)
    
    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 5. dim1 : 144.0, dim2 : 176.0
    Frame rate : 15.0 

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 6. dim1 : 120.0, dim2 : 160.0
    Frame rate : 15.0 

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 7. dim1 : 480.0, dim2 : 640.0
    Frame rate : 15.0 

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 8. dim1 : 480.0, dim2 : 640.0
    Frame rate : 15.0 
    
    (...)
    
More informations about plotting with matplotlib : https://matplotlib.org/users/pyplot_tutorial.html   
More informations about plot() : https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.plot

-  [X] Building a plot comparing the framerate with the downscale parameter

The plot construction is simple. It introduce two lists that get completed during the program execution :

    (...)
    downscales = []
    frames = []
    (...)
        frames.append(fps)
        downscales.append(ds)
        (...)
        
That get called by a few lines :

    plt.plot(downscales, frames)
    plt.title('Effect of downscale on framerate')
    plt.xlabel('Downscale')
    plt.ylabel('FPS')
    plt.show()

The program correctly shows the plot but it doesn't give any useful information, because the framerate doesn't seems influenced by the downscale.  
I tried to obtain more detailed informations, so I modified a few lines. Going from : 

    (...)
    for ds in range(1,7):
        (...)
        fps = nb_frames // seconds
        (...)
        
to :

    (...)
    for ds in np.arrange(1,7,0.5): #Modification needed because range() doesn't use floats, but np.arange() do
        (...)
        fps = nb_frames / seconds
        (...)
        
After some tries, it seems that the framerate is hightly test variable but it almost not influenced by the downscale.
        
I need to check with L. Perrinet if everything in the program is alright before going on the next steps.  

# 2017-05-03 : beggining the distant communication program
------------------------

I modified the program to integrate the framerate grabbing into the grab() function :

    def grab(self, nb_frames) :
            self.start = time.time()

            for nb in range(nb_frames):
                ret, frame_bgr = self.cap.read()
                #frame = frame_bgr[:, :, ::-1] #BGR to RBG.
                frame = frame_bgr

                self.end = time.time()
                self.seconds = self.end - self.start

                fps = nb_frames / self.seconds
                print ('Framerate :', fps,'\n')

                return frame, fps
                
That allowed me to make the calling block simplier :

    nb_frames = 50
    downscales_a, frames_a = [], []
    for ds in np.arange(1,7,0.5):

        Ph = PhotoReceptor(w=1280, h=720, DOWNSCALE = ds)

        frame, fps = Ph.grab(nb_frames)

        Ph.close()

        frames_a.append(fps)
        downscales_a.append(ds)
    
But the difference seems to have unbelievably speed up the framerate, going from about 16 FPS to more than 70 FPS. There must be an error in the framerate computation. Returning to the last checkpoint.

- [X] Etablishing a communication between the program and a distant camera

The communication must contain 3 things :  
1) A way to ask the camera if it's ready to record  
2) A way for the camera to respond positively or negatively  
3) A way to cut down the communication  

The communication can be etablished through a program that'll look like this pseudocode :

    __init__
    while True:
        grab()
        code
        send/recieve
    close()
    
More informations about the picamera library, allowing to communicate with a camera connected to a Raspberry Pi : https://github.com/laurentperrinet/openRetina/blob/master/src/openRetina.py  
Complete documentation : http://picamera.readthedocs.io/en/release-1.13/  
Note that may be important if recording a video or using a long exposure time : the picamera use a rolling shutter (vs global shutter, more informations : https://en.wikipedia.org/wiki/Rolling_shutter)  
A lot of informations about the current objective is available here : http://picamera.readthedocs.io/en/release-1.13/recipes1.html#capturing-to-a-network-stream  

The program must be divided in two scripts : one on the RaspBerry Pi that'll be used as a client, the other on a computer that'll be used as a server.  
The client will first send a 32-bit integer containing the length of the image that'll follow. If this length is 0, this indicate that no more images will be sent and the connection is shuted down.

    (...)
    connection = server_socket.accept()[0].makefile('rb')
    (...)
    
A problem seem to be provoked by the previous codeline. The program seem unable to go beyond it.  
More informations about the socket library : https://docs.python.org/2/library/socket.html  
After dividing the line into two :

    (...)
    connection = server_socket.accept()[0]
    connection.makefile('rb')
    (...)

It seems that the problem come from the first line, as a keyboard interrupt output the following error :

    (...)
    ---> 23 connection = server_socket.accept()
    (...)
    /usr/lib/python2.7/socket.pyc in accept(self)
        204 
        205     def accept(self):
    --> 206         sock, addr = self._sock.accept()
    (...)
    
The problem may come from the fact that no connection is etablished, because the client script is not writen et/or executed at this point.  
More informations about the struct library : https://docs.python.org/2/library/struct.html
More informations about the IO library : https://docs.python.org/2/library/io.html

# 2017-05-04 : networking and operating a RaspBerry Pi
----------------

Trying to solve some problems :

The graphics shows that there is no frame rate difference when applying a resolution downscale. This may come from a problem when defining the new camera resolution. Some of the downscales may not work because the resolutions we try to apply aren't supported by the camera/its driver.  
I'm looking for the most common resolutions to apply it to the program, instead of semi-randomly founding them with a np.arange() function.

- [X] Finding the right camera resolutions to apply as downscales

A list of the most common ones is available here : https://en.wikipedia.org/wiki/List_of_common_resolutions  
The 1280×720 native resolution of the camera display a pixel number equivalent to listed on the previous link. 
The camera support lot of resolutions (w x h, downscale in comparison of the native resolution), as : 
- 1280x720 (native)
- 1024x576 (downscale = 1.25)
- 720x480 (downscale = 1.5)
- 640x480 (downscale = ? and >=6.5)
- 352x288 (downscale = 2)
- 320x240 (downscale = 3)
- 176x144 (downscale = 4)
- 160x120 (downscale = 6)

I don't find the exact downscale parameter to apply a 720x480 resolution, as it seems to goes directly from 1024x576 to 352x288, even with a step = 0.2.  
Still, __no frame rate difference is notable when applying a downscale.__

- [X] Search more informations about the cv.SetCaptureProperty(capture, property_id, value) function

Some informations available here : http://docs.opencv.org/2.4/modules/highgui/doc/reading_and_writing_images_and_video.html#videocapture-set
The following functions : 

    self.cap.set(cv2.cv.CV_CAP_PROP_FRAME_WIDTH, int(W/self.DOWNSCALE))
    self.cap.set(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT, int(H/self.DOWNSCALE))
    
Could have only modified the screen size displayed and not the recording proprieties, so this function :

    cv.SetCaptureProperty(capture, property_id, value)
    
Found on the OpenCv documentation could have been interesting. But it seems to be outdated and only supporting video files.  

- [X] Connecting the computer and the RaspBerry

The RaspBerry is connected to internet through an ethernet cable and have an IPv4 adress : __10.165.5.117__

    #Client
    ssh pi@10.165.5.117
    
    The authenticity of host '10.164.5.117 (10.164.5.117)' can't be established.
    ECDSA key fingerprint is SHA256:20dejdkEy0IVAnn8gfBZqzoX/uo7qd/ZvMVyFeBEFcc.
    Are you sure you want to continue connecting (yes/no)? yes
    Warning: Permanently added '10.164.5.117' (ECDSA) to the list of known hosts.
    
    pi@10.164.5.117's password: _____________________________________

    The programs included with the Debian GNU/Linux system are free software;
    the exact distribution terms for each program are described in the
    individual files in /usr/share/doc/*/copyright.

    Debian GNU/Linux comes with ABSOLUTELY NO WARRANTY, to the extent
    permitted by applicable law.
    Last login: Fri Mar  3 16:48:39 2017 from int-users-5-201.intlocal.univ-amu.fr
    
Getting some informations about the device I just connected to :

    #Client
    uname -a
    Linux retina 4.4.50-v7+ #970 SMP Mon Feb 20 19:18:29 GMT 2017 armv7l GNU/Linux

    
- [X] Writing the script that'll be executed by client

The server script should be run first, to be sure there's a server socket listening the connection.  

- [X] Adding a ssh-key to the client

I have the possibility to insert a ssh key to the raspbian to bypass the password requirement. Since I already have an ssh key that I created to connect github through git, I'm trying to add that ssh key to the raspberry .ssh/autorized_keys.  
More informations : https://www.raspberrypi.org/documentation/remote-access/ssh/passwordless.md

    #Host
    cat ~/.ssh/id_rsa.pub | ssh pi@10.164.5.117 'cat >> .ssh/authorized_keys'
    pi@10.164.5.117's password: ____________________
    
Then verifying that the ssh key was correctly sent to the raspberry : 

    #Client
    cat ~/.ssh/authorized_keys
    ssh-key <key1> <id1>
    ssh-key <key2> <id2>
    
Installing some libraries on the pi :
    
    sudo aptitude install python3-pip
    pip install -e
    
- [X] Taking a picture with raspistill

To familiarize myself with operating on the pi, I try to take a picture, save it and manipulate it throught raspistill (basic tool for interacting with the camera module, more informations : https://www.raspberrypi.org/documentation/usage/camera/raspicam/raspistill.md).  
More informations about the camera (hardware) : https://www.raspberrypi.org/documentation/hardware/camera/README.md  
Simple command line to better understand raspistill : 

    #Client
    raspistill --help
    
Taking a picture is really simple : 

    #Client
    raspistill -o firstpic.png
    
- [X] Preparing the client to operate
    
Retrieve the github documents and checking everything got transfered : 

    #Client
    git clone https://github.com/pierrealbiges/INT-internship
    ls ~/INT-internship/
    
Had to install some python libraries, including picamera, to  make the client run the scripts.

- [X] Running 2017_04_0*_networking_clientside.py

When running the script, I get the following error as an output : 
    
    #Client
    (...)
    --> client_socket.connect(('',8000))
    File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
    socket.error: [Errno 111] Connection refused
    
Trying to add the computer IPv4, so the previous bugged line become :

    client_socket.connect(('10.164.6.47',8000))
    
It works ! The server side is outputing :

    Image is 640480 pixels
    Image is verified 

Numerous times, until :

    Image length missing or zero
    Connection closed
    
But the server script doesn't show the image sent by the client script. After running enterily, it output an error :

    (...)
    ---> 24         image = Image.open(image_stream)
    (...)
     2316     raise IOError("cannot identify image file %r"
    -> 2317                   % (filename if filename else fp))
    (...)
    IOError: cannot identify image file <_io.BytesIO object at 0x7ff9232c3e30>

Updraging PIL version may help (as mentionned here : http://stackoverflow.com/questions/19230991/image-open-cannot-identify-image-file-python) :

    pip install --upgrade pillow
    
After upgrading, the same error is still showing.

# 2017-05-05 : showing pictures taken by the raspberry and creating a daemon
---

Is the client ssh on? ssh connections are outputing a 'timed out' error. Everything worked fine when leaving the lab yesterday.  
No response from the ping command. Is the pi on?

    #Client
    ping 10.165.5.117
    PING 10.165.5.117 (10.165.5.117) 56(84) bytes of data.

    --- 10.165.5.117 ping statistics ---
    199 packets transmitted, 0 received, 100% packet loss, time 202745ms
    
Restarting the pi seemed to solve the problem.

- [X] Solving the Image.open() error

It seems that PIL (the library that contain Image) was not updated since 2009 (see : http://www.pythonware.com/products/pil/). If I don't find a solution under PIL, it may be intersting to migrate to Pillow, a PIL friendly fork that was last updated in 2016 (see : http://pillow.readthedocs.io/en/4.1.1/index.html).  
It seems that the migration was already done, as 

    #Host
    pip list
    
output : 

    (...)
    Pillow (3.3.1)
    (...)
    
More informations about the Pillow's Image module : http://pillow.readthedocs.io/en/3.1.x/reference/Image.html  
Since the command :

    #Host
    image.verify()
    
Doesn't output any error, the file must be correclty encoded. The problem must come from the reading step of the image.
Adding the image.show() command line to the script allowed to show all the images captured by the pi camera : 

    #Host
    (...)
    image = Image.open(image_stream)
    image.show()
    (...)
    
The Image.open() error is still showing, but as all the images are correclty showing, it may be triggered by the last bytes string which serves as a closing message.  
To avoid the error message to show, I modify the script, going from :

    #Host
    (...)
    connection = True
    while connection = True:
        (...)
            (...)
            connection = False

to :

    #Host
    (...)
    while True:
        (...)
            (...)
            break
            
- [ ] Integrate networking_serverside to camera.py : Reported

Going back to the client/server communication.  
The communication must contain :  
1) A way to ask the camera if it's ready to record  
2) A way for the camera to respond positively or negatively  
3) A way to cut down the communication  
I need to create a daemon : a program that run as a background process (more informations : https://en.wikipedia.org/wiki/Daemon_(computing) ). 
I'll then write an API (more informations : https://en.wikipedia.org/wiki/Application_programming_interface) to communicate with the client/daemon through the python libraries zmq and socket.

- [ ] Create a daemon

Daemons usually have a 'd' at the end of their name to show rightly that they are one (ex: sshd for the ssh process). Since ours is needed for the OpenRetina manipulation, it could be called __retinad__.

They are usually started at boot (by the system init) to respond to any request they are programmed for. It cannot be launched twice during a session.  
It must/be able to : 
- be dissociated from the console control
- be a session and process group leader(?) 
- be executed as a background task (usually by forking aka creating a copy of itself, more informations : https://en.wikipedia.org/wiki/Fork_(system_call) then exiting, more informations : https://en.wikipedia.org/wiki/Exit_(system_call))
- change te umask (more information : https://en.wikipedia.org/wiki/Umask) to grant its subprocess (some or all of them) the right to read, write and execute files.
- close/open its subprocess
- and others... (see : https://www.python.org/dev/peps/pep-3143/#correct-daemon-behaviour)  
Basically, any program can be turned as daemon if it follows these rules. The quality of it behavior depend of how much it follow them, in addition of the security ones (preventing the generation of core files, for example).  

Services differ from daemon because they are programs that start an additional process that'll run in background, instead of the actual program.

Lot of implementations are already existing (see : https://www.python.org/dev/peps/pep-3143/#other-daemon-implementations, lot of them are updates of a daemon called stevens).  
Two of them looks interesting and are still updated : https://pypi.python.org/pypi/python-daemon/ and http://supervisord.org/. Supervisored documention seems way more abundant; but python-daemon seems to be used more often.  
More informations about python-daemon and its use and history : http://stackoverflow.com/questions/30408589/how-do-you-use-python-daemon-the-way-that-its-documentation-dictates  

(More informations about 

    __name__ = "__main__"
    
here : http://stackoverflow.com/questions/419163/what-does-if-name-main-do)  
More informations about stdin, stdout and stderr : https://en.wikipedia.org/wiki/Standard_streams  

As generalization and free-using is one of the objectives of OpenRetina, Supervisord can't be used because it does not run under python v.>3. and Windows, and need a constant internet connection.

A whole daemon is not necessary. A python script running in the background and available for interactions will do the job.  
A python script can run in background, even if the terminal that called it is closed, using the nohup command (more informations : https://askubuntu.com/questions/396654/how-to-run-the-python-program-in-the-background-in-ubuntu-machine and http://manpages.ubuntu.com/manpages/precise/man1/nohup.1.html).

    nohup path/to/script.py &
    
To see the process again :
    
    ps ax | grep script.py